In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import mwparserfromhell as mw
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2
import processing # this makes the "processing" folder available as a python module

from features.combatants import get_combatant, get_combatants
from utils import remove_ref, get_templates

In [ ]:
battles = json.load(open("../datasets/battle-fields-0.json"))
battles_ib = [b["infobox"] for b in battles if not b["infobox"].get("error")]
comb_parsed = [get_combatants(b) for b in battles if not b["infobox"].get("error")]
comb = [{"combatant_%s_source" % n: b.get("combatant%s"%n)  for n in range(1,4)} for b in battles_ib]
for cp, c in zip(comb_parsed, comb):
    cp.update(c)

In [ ]:
print("tot", len(comb_parsed))
df = pd.DataFrame(comb_parsed).fillna(np.nan)
df.isna().sum()

In [ ]:
df[df["combatant_1"].isna()]

In [ ]:
print(mw.parse("{{flagicon|Bad Guy inc}}[[Bad Guy|Bad Guy Verbose]] supported by [[OK Guy|OK-ish Guy Verbose]] (BG)<ref name=\"stamfordhistory.org\">[http://www.stamfordhistory.org/ww2_peleliu.htm The Stamford Historical Society: Peleliu] Retrieved 27 Oct. 2015</ref><br>").strip_code())
get_combatant("{{flagicon|Bad Guy inc}}[[Bad Guy|Bad Guy Verbose]] supported by [[OK Guy|OK-ish Guy Verbose]] (BG)<ref name=\"stamfordhistory.org\">[http://www.stamfordhistory.org/ww2_peleliu.htm The Stamford Historical Society: Peleliu] Retrieved 27 Oct. 2015</ref><br>")

In [ ]:
cbt_1 = np.array(df["combatant_1"].dropna())
cbt_2 = np.array(df["combatant_2"].dropna())
cbt_3 = np.array(df["combatant_3"].dropna())
print(len(cbt_1), len(cbt_2), len(cbt_3))

cbt_all = np.concatenate((cbt_1, cbt_2, cbt_3))

all_cbt_names = [c for cl in cbt_all for c in cl]
names = pd.Series(all_cbt_names)
print(len(set(all_cbt_names)))

f, ax = plt.subplots(figsize=(6, 15))
counts = names.value_counts().sort_values(ascending=False)
counts = counts.head(50)
sns.barplot(x=counts, y=counts.index, ax=ax)
counts